# Clean products data

## Load products data that we scraped before

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re    
import pickle

In [1]:
product_df = pd.read_csv('product_info_raw.csv', index_col = 0)
product_df.head()

/home/ktgiahieu/env/ds/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,sku,name,url_key,url_path,type,book_cover,short_description,price,list_price,...,cross_border_policy_text,dia_chi_su_dung,expiry_date,only_ship_to_nested,has_other_fee,publisher,authors,is_baby_milk,is_acoholic_drink,confirm_over_age
0,21299202,2900407073325,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,cap-sac-va-truyen-du-lieu-toc-do-cao-baseus-ho...,cap-sac-va-truyen-du-lieu-toc-do-cao-baseus-ho...,configurable,NaN,Tính năng cơ bản Cáp sạc và truyền dữ liệu tốc...,67000,160000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44105732,2348519488017,"Dép sục unisex duet độn đế, chống trượt Litic ...",dep-suc-unisex-duet-don-de-chong-truot-litic-l...,dep-suc-unisex-duet-don-de-chong-truot-litic-l...,configurable,NaN,"\nDép sục unisex duet độn đế, chống trượt Liti...",148500,280000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54132740,2009091688508,bình thông hơi dành cho xe máy tặng bao tay xe...,binh-thong-hoi-danh-cho-xe-may-tang-bao-tay-xe...,binh-thong-hoi-danh-cho-xe-may-tang-bao-tay-xe...,configurable,NaN,bình thông hơi dành cho xe máy tặng bao tay xe...,199000,220000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,65011722,6564088454154,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",dong-ho-nam-fngeen-fe230-mat-dong-ho-tron-thie...,dong-ho-nam-fngeen-fe230-mat-dong-ho-tron-thie...,configurable,NaN,\nVới sản phẩm đồng hồ nam thời trang FNGEEN (...,179000,500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7995403,2886417334581,Giày Sneaker Unisex Converse Chuck Taylor All ...,giay-sneaker-unisex-converse-chuck-taylor-all-...,giay-sneaker-unisex-converse-chuck-taylor-all-...,configurable,NaN,Chất liệu vải cao cấp\nGiày Sneaker Unisex Con...,1500000,1500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Select relevant features

In [6]:
product_df_clean = product_df.copy()
from ast import literal_eval
product_df_clean['category'] = product_df['categories'].apply(lambda x:literal_eval(x)['id'])
product_df_clean['category_name'] = product_df['productset_group_name']
features = ['id', 'name', 'review_count', 'rating_average', 'price', 'description', 'category', 'category_name']
product_df_clean = product_df_clean[features]
product_df_clean.head()

,id,name,review_count,rating_average,price,description,category,category_name
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,<span class='description-separate'></span>Hỗ t...,1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,<span class='description-separate'></span>Tiện...,914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,<span class='description-separate'></span>bình...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,<span class='description-separate'></span>Mặt ...,914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,<span class='description-separate'></span>Đườn...,49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...


Next, we need to parse html description of the data to extract the description we need

In [7]:
# We are gonna use BeautifulSoup to parse the data
from bs4 import BeautifulSoup as bs
def extract_description(content):
    soup = bs(content)
    return soup.get_text(" ", strip = True).replace('\xa0', ' ')
product_df_clean['description'] = product_df_clean['description'].apply(extract_description)
product_df_clean['description'].head()

0    Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...
1    Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...
2    bình thông hơi dành cho xe máy tặng bao tay xe...
3    Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...
4    Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...
Name: description, dtype: object

### Cleaned product data

In [8]:
product_df_clean.head()

,id,name,review_count,rating_average,price,description,category,category_name
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,"Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...",914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,bình thông hơi dành cho xe máy tặng bao tay xe...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,"Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...",914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,"Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...",49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...


In [10]:
product_df_clean.to_csv('product_info_clean.csv')

# Clean reviews data

## Load reviews data

In [2]:
review_df = pd.read_csv('product_review_raw.csv', index_col = 0)
review_df.head()

/home/ktgiahieu/env/ds/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,title,content,status,thank_count,score,new_score,customer_id,comment_count,rating,...,thanked,created_at,created_by,suggestions,attributes,spid,seller,product_id,comments,content_id
0,4607425,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,approved,6,0.025914,0.128174,5719606,0,5,...,False,1598339055,"{'id': None, 'name': 'Khách Hàng', 'full_name'...",[],"['Mua từ nhà bán Tổng Kho Phụ Kiện Số HD', 'Mà...",51957764.0,"{'id': 113024, 'name': 'Tổng Kho Phụ Kiện Số H...",21299202,[],NaN
1,4428169,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",approved,19,0.023013,0.116557,11891537,2,1,...,False,1597207301,"{'id': 11891537, 'name': 'Trung Tung Tăng', 'f...",[],"['Mua từ nhà bán Tổng Kho Phụ Kiện Số HD', 'Mà...",51957757.0,"{'id': 113024, 'name': 'Tổng Kho Phụ Kiện Số H...",21299202,"[{'id': '743929', 'review_id': '4428169', 'cus...",NaN
2,4731909,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",approved,0,22.222222,0.114341,7630980,0,4,...,False,1599192921,"{'id': 7630980, 'name': 'Nguyễn Tất Thắng', 'f...",[],"['Mua từ nhà bán Tổng Kho Phụ Kiện Số HD', 'Mà...",51957757.0,"{'id': 113024, 'name': 'Tổng Kho Phụ Kiện Số H...",21299202,[],NaN
3,4474171,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",approved,0,0.790297,0.109959,1242105,1,1,...,False,1597475983,"{'id': 1242105, 'name': 'Doan Hiep', 'full_nam...",[],"['Mua từ nhà bán Tổng Kho Phụ Kiện Số HD', 'Mà...",51957764.0,"{'id': 113024, 'name': 'Tổng Kho Phụ Kiện Số H...",21299202,"[{'id': '748580', 'review_id': '4474171', 'cus...",NaN
4,3121561,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,approved,8,0.013937,0.108435,10815590,0,4,...,False,1583648581,"{'id': 10815590, 'name': 'Hoàng việt hưng', 'f...",[],"['Mua từ nhà bán PKS Dragon', 'Màu sắc Đỏ', 'C...",44223585.0,"{'id': 88455, 'name': 'PKS Dragon', 'logo': 'h...",21299202,[],NaN


## Select relevant features

In [3]:
features = ['customer_id', 'product_id', 'rating', 'thank_count', 'title', 'content', 'created_at', 'created_by']
review_df_clean = review_df[features]
review_df_clean.head()

,customer_id,product_id,rating,thank_count,title,content,created_at,created_by
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,1598339055,"{'id': None, 'name': 'Khách Hàng', 'full_name'..."
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",1597207301,"{'id': 11891537, 'name': 'Trung Tung Tăng', 'f..."
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",1599192921,"{'id': 7630980, 'name': 'Nguyễn Tất Thắng', 'f..."
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",1597475983,"{'id': 1242105, 'name': 'Doan Hiep', 'full_nam..."
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,1583648581,"{'id': 10815590, 'name': 'Hoàng việt hưng', 'f..."


Next, we need to parse data from **created_by** column

In [15]:
from ast import literal_eval
def extract_purchase_time(x):
    try:
        return literal_eval(x)['purchased_at']
    except:
        return np.nan
def extract_customer_name(x):
    try:
        return literal_eval(x)['full_name']
    except:
        return np.nan    

review_df_clean['purchase_at'] = review_df_clean['created_by'].apply(extract_purchase_time)
review_df_clean['customer_name'] = review_df_clean['created_by'].apply(extract_customer_name)
review_df_clean.head()

/home/ktgiahieu/env/ds/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/ktgiahieu/env/ds/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,product_id,rating,thank_count,title,content,created_at,created_by,purchase_at,customer_name
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,1598339055,"{'id': None, 'name': 'Khách Hàng', 'full_name'...",1.593585e+09,Khách Hàng
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",1597207301,"{'id': 11891537, 'name': 'Trung Tung Tăng', 'f...",1.596726e+09,Trung Tung Tăng
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",1599192921,"{'id': 7630980, 'name': 'Nguyễn Tất Thắng', 'f...",1.599058e+09,Nguyễn Tất Thắng
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",1597475983,"{'id': 1242105, 'name': 'Doan Hiep', 'full_nam...",1.597327e+09,Doan Hiep
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,1583648581,"{'id': 10815590, 'name': 'Hoàng việt hưng', 'f...",1.582278e+09,Hoàng việt hưng


### Convert timestamp into DateTime format

In [20]:
from datetime import datetime
def extract_datetime(x):
    try:
        return datetime.fromtimestamp(x)
    except:
        return np.nan    
review_df_clean['created_at'] = review_df_clean['created_at'].apply(extract_datetime)
review_df_clean['purchase_at'] = review_df_clean['purchase_at'].apply(extract_datetime)

In [21]:
review_df_clean.head()

,customer_id,product_id,rating,thank_count,title,content,created_at,purchase_at,customer_name
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,2020-08-25 14:04:15,2020-07-01 13:22:12,Khách Hàng
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",2020-08-12 11:41:41,2020-08-06 22:05:10,Trung Tung Tăng
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",2020-09-04 11:15:21,2020-09-02 21:39:54,Nguyễn Tất Thắng
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",2020-08-15 14:19:43,2020-08-13 20:50:11,Doan Hiep
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,2020-03-08 13:23:01,2020-02-21 16:38:52,Hoàng việt hưng


### Cleaned product data

In [25]:
review_df_clean.drop(columns = ['created_by'], inplace=True)

In [26]:
review_df_clean.head()

,customer_id,product_id,rating,thank_count,title,content,created_at,purchase_at,customer_name
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,2020-08-25 14:04:15,2020-07-01 13:22:12,Khách Hàng
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",2020-08-12 11:41:41,2020-08-06 22:05:10,Trung Tung Tăng
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",2020-09-04 11:15:21,2020-09-02 21:39:54,Nguyễn Tất Thắng
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",2020-08-15 14:19:43,2020-08-13 20:50:11,Doan Hiep
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,2020-03-08 13:23:01,2020-02-21 16:38:52,Hoàng việt hưng


In [22]:
review_df_clean.to_csv('product_review_clean.csv')